# [문장 유형 분류 AI 경진대회](https://dacon.io/competitions/official/236037/overview/description)

## Dataset Info.

+ train.csv [파일]
    - ID : 샘플 문장 별 고유 ID
    - 문장 : 샘플 별 한개의 문장
    - 유형 : 문장의 유형 (사실형, 추론형, 대화형, 예측형)
    - 극성 : 문장의 극성 (긍정, 부정, 미정)
    - 확실성 : 문장의 확실성 (확실, 불확실)
    - label : 문장 별 유형, 극성, 시제, 확실성에 대한 Class (총 72개 종류의 Class 존재)  
    예시) 사실형-긍정-현재-확실


- test.csv [파일]
    + ID : 샘플 문장 별 고유 ID
    + 문장 : 샘플 별 한개의 문장


+ sample_submission.csv [파일] - 제출 양식
    - ID : 샘플 문장 별 고유 ID
    - label : 예측한 문장 별 유형, 극성, 시제, 확실성에 대한 Class  
    예시) 사실형-긍정-현재-확실


In [1]:
test_title = "T00_Baseline"

## Code

### 01_Import

In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

/opt/miniconda3/envs/pytorch_gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

PyTorch version:1.13.0.dev20220913
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-12.5-arm64-arm-64bit


### 02_Hyperparameter Setting

In [4]:
CFG = {
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 256,
    'SEED': 190813
}

### 03_Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(CFG['SEED']) # Seed 고정

### 04_Data Load

In [6]:
df = pd.read_csv('./01_data/train.csv')
test = pd.read_csv('./01_data/test.csv')

### 05_Train / Validation Split

In [7]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

### 06_Data Pre-processing

#### A_문장(Text) 벡터화 via TfidfVectorizer

In [8]:
vectorizer = TfidfVectorizer(min_df=4, analyzer='word', ngram_range=(1, 2))
vectorizer.fit(np.array(train['문장']))

train_vec = vectorizer.transform(train['문장'])
val_vec = vectorizer.transform(val['문장'])
test_vec = vectorizer.transform(test['문장'])

print(train_vec.shape, val_vec.shape, test_vec.shape)

(13232, 9295) (3309, 9295) (7090, 9295)


#### B_Label Encoding (`유형`, `극성`, `시제`, `확실성`)

In [9]:
type_le = preprocessing.LabelEncoder()
train['유형'] = type_le.fit_transform(train['유형'].values)
val['유형'] = type_le.transform(val['유형'].values)

polarity_le = preprocessing.LabelEncoder()
train['극성'] = type_le.fit_transform(train['극성'].values)
val['극성'] = type_le.transform(val['극성'].values)

tense_le = preprocessing.LabelEncoder()
train['시제'] = type_le.fit_transform(train['시제'].values)
val['시제'] = type_le.transform(val['시제'].values)

certainty_le = preprocessing.LabelEncoder()
train['확실성'] = type_le.fit_transform(train['확실성'].values)
val['확실성'] = type_le.transform(val['확실성'].values)

In [10]:
train_type = train['유형'].values
train_polarity = train['극성'].values
train_tense = train['시제'].values
train_certainty = train['확실성'].values

train_labels = {
    'type': train_type,
    'polarity': train_polarity,
    'tense': train_tense,
    'certainty': train_certainty
}

In [11]:
val_type = val['유형'].values
val_polarity = val['극성'].values
val_tense = val['시제'].values
val_certainty = val['확실성'].values

val_labels = {
    'type': val_type,
    'polarity': val_polarity,
    'tense': val_tense,
    'certainty': val_certainty
}

### 07_Custom Dataset

In [12]:
class CustomDataset(Dataset):
    def __init__(self, st_vec, st_labels):
        self.st_vec = st_vec
        self.st_labels = st_labels
        
    def __getitem__(self, index):
        st_vector = torch.FloatTensor(self.st_vec[index].toarray()).squeeze(0)
        if self.st_labels is not None:
            st_type = self.st_labels['type'][index]
            st_polarity = self.st_labels['polarity'][index]
            st_tense = self.st_labels['tense'][index]
            st_certainty = self.st_labels['certainty'][index]
            return st_vector, st_type, st_polarity, st_tense, st_certainty
        else:
            return st_vector
        
    def __len__(self):
        return len(self.st_vec.toarray())

In [13]:
train_dataset = CustomDataset(train_vec, train_labels)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_vec, val_labels)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 08_Model Define

In [14]:
class BaseModel(nn.Module):
    def __init__(self, input_dim=9351):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
        )
        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=4),
        )
        self.polarity_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=3)
        )
        self.tense_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=3)
        )
        self.certainty_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=2)
        )
    
    def forward(self, x):
        x = self.feature_extract(x)
        # 문장 유형, 극성, 시제, 확실성을 각각 분류
        type_output = self.type_classifier(x)
        polarity_output = self.polarity_classifier(x)
        tense_output = self.tense_classifier(x)
        certainty_output = self.certainty_classifier(x)
        return type_output, polarity_output, tense_output, certainty_output

### 09_Train

In [15]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    criterion = {
        'type': nn.CrossEntropyLoss().to(device),
        'polarity': nn.CrossEntropyLoss().to(device),
        'tense': nn.CrossEntropyLoss().to(device),
        'certainty': nn.CrossEntropyLoss().to(device)
    }
    
    best_loss = 999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for sentence, type_label, polarity_label, tense_label, certainty_label in tqdm(iter(train_loader)):
            sentence = sentence.to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)
                        
            optimizer.zero_grad()
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            loss = 0.25 * criterion['type'](type_logit, type_label) + \
                0.25 * criterion['polarity'](polarity_logit, type_label) + \
                0.25 * criterion['tense'](tense_logit, type_label) + \
                0.25 * criterion['certainty'](certainty_logit, type_label)
                
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
        val_loss, val_type_f1, val_polarity_f1, val_tense_f1, val_certainty_f1 = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}]   Train Loss : [{np.mean(train_loss):.5f}]   Val Loss : [{val_loss:.5f}] 유형 F1: [{val_type_f1:.5f}] 극성 F1 : [{val_polarity_f1:.5f}] 시제 F1 : [{val_tense_f1:.5f}] 확실성 F1 : [{val_certainty_f1:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_loss)
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
        
        return best_model

In [16]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    type_labels, polarity_labels, tense_labels, certainty_labels = [], [], [], []
    
    with torch.no_grad():
        for sentence, type_label, polarity_label, tense_label, certainty_label in tqdm(iter(val_loader)):
            sentence = sentence.to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            loss = 0.25 * criterion['type'](type_logit, type_label) + \
                0.25 * criterion['polarity'](polarity_logit, type_label) + \
                0.25 * criterion['tense'](tense_logit, type_label) + \
                0.25 * criterion['certainty'](certainty_logit, type_label)
                
            val_loss.append(loss.item())
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            type_labels += type_label.detach().cpu().numpy().tolist()
            
            polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            polarity_labels += polarity_label.detach().cpu().numpy().tolist()
            
            tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            tense_labels += tense_label.detach().cpu().numpy().tolist()
            
            certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            certainty_labels += certainty_label.detach().cpu().numpy().tolist()
    
    type_f1 = f1_score(type_labels, type_preds, average='weighted')
    polarity_f1 = f1_score(polarity_labels, polarity_preds, average='weighted')
    tense_f1 = f1_score(tense_labels, tense_preds, average='weighted')
    certainty_f1 = f1_score(certainty_labels, certainty_preds, average='weighted')
    
    return np.mean(val_loss), type_f1, polarity_f1, tense_f1, certainty_f1

### 09_Run

In [17]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/52 [00:00<?, ?it/s]

: 

: 

### 10_Inference

In [ ]:
test_dataset = CustomDataset(test_vec, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    
    with torch.no_grad():
        for sentence in tqdm(test_loader):
            sentence = sentence.to(device)
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
        
    return type_preds, polarity_preds, tense_preds, certainty_preds

In [ ]:
type_preds, polarity_preds, tense_preds, certainty_preds = inference(model, test_loader, device)

In [ ]:
type_preds = type_le.inverse_transform(type_preds)
polarity_preds = polarity_le.inverse_transform(polarity_preds)
tense_preds = tense_le.inverse_transform(tense_preds)
certainty_preds = certainty_le.inverse_transform(certainty_preds)

In [ ]:
predictions = []
for type_pred, polarity_pred, tense_pred, certainty_pred in zip(type_preds, polarity_preds, tense_preds, certainty_preds):
    predictions.append(type_pred+'-'+polarity_pred+'-'+tense_pred+'-'+certainty_pred)

### 11_Submission

In [ ]:
submit = pd.read_csv('./01_data/sample_submission.csv')
submit['label'] = predictions

In [ ]:
submit.head()

In [ ]:
submit.to_csv('./02_submission/{test_title}_sumbit.csv', index=False)